<a href="https://colab.research.google.com/github/dahlia52/Commercial-District-Analysis/blob/main/code/%EC%A3%BC%EA%B3%A0%EA%B0%9D%EB%B0%8F%EC%97%85%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
### 발달상권에 해당하는 경우만 보자

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import numpy as np

In [ ]:
sales = pd.read_csv("./전처리 완료/sales_광진.csv")
hotplace_standard = pd.read_csv("발달상권 클러스터.csv")
market_standard = pd.read_csv("전통시장 클러스터.csv")
nearby_standard = pd.read_csv("골목상권 클러스터.csv")

In [ ]:
hotplace = sales[sales['상권_구분_코드_명'] == '발달상권'].reset_index(drop=True) #광진구 내 발달상권
market = sales[sales['상권_구분_코드_명'] == '전통시장'].reset_index(drop=True) #광진구 내 전통시장
nearby = sales[sales['상권_구분_코드_명'] == '골목상권'].reset_index(drop=True) #광진구 내 골목상권

In [ ]:
def store_service(raw, num): #구찮아서 함수 만듦...
    df = raw[raw['기준_분기_코드'] == num].copy()
    df = df.groupby(['상권_코드_명', '서비스_업종_코드_명']).agg({'분기당_매출_금액': 'sum'}).reset_index(drop=False)

    # 상권은 행, 업종은 컬럼, 매출 금액은 value
    df = df.pivot(index='상권_코드_명', columns='서비스_업종_코드_명', values='분기당_매출_금액')
    df.fillna(0, inplace=True)
    df['전체 매출'] = df.sum(axis=1, numeric_only=True) #상권의 전체 매출 계산

    col = list(df.columns)
    for c in col: # ratio 열 생성
        if c != "전체 매출":
            df[f'ratio_{c}'] = df[c] / df['전체 매출'] * 100

    col = list(df.columns)
    for c in col: # ratio 컬럼만 저장
        if "ratio" not in c:
            df.drop([c], axis=1, inplace=True)

    df.reset_index(drop=False, inplace=True)
    df['매출 최대 비율'] = df.max(axis=1, numeric_only=True)
    df['매출 최대 업종'] = np.nan

    col = list(df.columns)
    for i in range(0,len(df)):
        for c in col:
            if ("매출 최대" not in c) and c != "상권_코드_명":
                if df.loc[i, c] == df.loc[i, '매출 최대 비율']: #매출 최대 비율에 해당한다면
                    df.loc[i, '매출 최대 업종'] = c #매출 최대 업종에 컬럼명 삽입

    for i,j in enumerate(df['매출 최대 업종']): #업종명만 남기기
        df.loc[i, '매출 최대 업종'] = j.split("_")[1]

    new_df = df[['상권_코드_명', '매출 최대 비율', '매출 최대 업종']]
    new_df.columns =['상권_코드_명', str(num)+"분기", str(num)+"분기 매출 최대 업종"]
    return new_df

In [ ]:
from functools import reduce

def concat_quart(raw, function):
    fourth = function(raw, 4)
    third = function(raw,3)
    second = function(raw,2)
    first = function(raw, 1)
    dfs = [first, second, third, fourth]
    df = reduce(lambda left, right: pd.merge(left, right, on='상권_코드_명'), dfs) # 상권코드명 기준으로 merge
    return df

In [ ]:
import dataframe_image as dfi

df1 = concat_quart(hotplace, store_service)
df1 = df1.merge(hotplace_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df1, "./clustering/발달상권 우세업종.png", max_cols=-1, max_rows=-1)
df1

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 매출 최대 업종,2분기,2분기 매출 최대 업종,3분기,3분기 매출 최대 업종,4분기,4분기 매출 최대 업종,label
0,구삼강변맛길,24.689038,슈퍼마켓,21.928467,슈퍼마켓,22.956442,슈퍼마켓,23.704242,슈퍼마켓,0
1,구의사거리,27.096354,철물점,20.886019,조명용품,18.530628,조명용품,34.724728,철물점,0
2,구의역,23.729103,한식음식점,25.457010,한식음식점,26.881040,한식음식점,28.128271,한식음식점,0
3,아차산역,14.248538,한식음식점,14.257295,일반의류,13.953105,한식음식점,15.680184,한식음식점,0
4,어린이대공원역,27.377293,한식음식점,27.484695,한식음식점,25.141445,한식음식점,26.157083,한식음식점,0
5,올림픽대교북단,18.541917,일반의원,19.268224,일반의원,17.526248,일반의원,21.252926,일반의원,0
6,강변역(테크노마트),48.002397,컴퓨터및주변장치판매,35.337696,컴퓨터및주변장치판매,38.192463,컴퓨터및주변장치판매,38.715883,컴퓨터및주변장치판매,1
7,건대입구역(건대),16.355954,한식음식점,18.663809,한식음식점,19.333606,한식음식점,20.416513,한식음식점,1
8,군자역,18.411707,한식음식점,19.957432,한식음식점,21.280810,한식음식점,23.016282,한식음식점,1


In [ ]:
df2 = concat_quart(nearby, store_service)
df2 = df2.merge(nearby_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df2, "./clustering/골목상권 우세업종.png", max_cols=-1, max_rows=-1)
df2

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 매출 최대 업종,2분기,2분기 매출 최대 업종,3분기,3분기 매출 최대 업종,4분기,4분기 매출 최대 업종,label
0,화양사거리,42.974948,편의점,37.805910,편의점,39.564172,편의점,35.135330,편의점,0
1,용마초등학교,25.523606,의약품,17.818880,의약품,21.108445,의약품,20.449651,의약품,0
2,자양4동주민센터,31.446867,반찬가게,18.434666,편의점,20.966290,편의점,22.798898,반찬가게,0
3,자양한강도서관,27.613762,한식음식점,30.057532,한식음식점,31.634608,한식음식점,36.421956,한식음식점,0
4,능동우편취급국,31.950226,편의점,24.549654,편의점,31.227580,편의점,30.577100,한식음식점,0
5,구의역 4번,15.125787,편의점,13.699731,편의점,15.856613,편의점,15.063622,한식음식점,0
6,구의1동주민센터,17.290012,애완동물,24.706188,호프-간이주점,20.987208,호프-간이주점,23.163889,한식음식점,0
7,중곡3동주민센터,16.873543,편의점,18.901884,일반의원,20.866962,편의점,18.062450,일반의원,0
8,아차산역 1번,19.634742,육류판매,19.317345,한식음식점,18.360033,육류판매,21.161101,한식음식점,0
9,광진정보도서관,21.399096,한식음식점,22.091874,한식음식점,24.086362,한식음식점,20.977869,한식음식점,0


In [ ]:
df3 = concat_quart(market, store_service)
df3 = df3.merge(market_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df3, "./clustering/전통시장 우세업종.png", max_cols=-1, max_rows=-1)
df3

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 매출 최대 업종,2분기,2분기 매출 최대 업종,3분기,3분기 매출 최대 업종,4분기,4분기 매출 최대 업종,label
0,노룬산시장(노룬산골목시장),21.502409,청과상,20.320822,청과상,21.261557,청과상,13.811926,청과상,0
1,능동로골목시장,33.026388,한식음식점,38.261386,한식음식점,32.744108,한식음식점,32.127768,한식음식점,0
2,면곡골목시장,56.605976,슈퍼마켓,29.967876,한식음식점,58.120635,슈퍼마켓,50.048136,슈퍼마켓,0
3,신성시장(신성골목시장),19.928356,슈퍼마켓,18.801680,슈퍼마켓,19.602323,슈퍼마켓,18.187735,슈퍼마켓,0
4,영동교골목시장,56.125168,육류판매,53.581217,육류판매,63.899264,육류판매,61.575391,육류판매,0
5,조양시장,52.084478,제과점,50.115303,치과의원,44.432602,제과점,46.645611,제과점,0
6,한아름시장,27.312075,일반의원,26.482160,일반의원,22.220824,분식전문점,24.374501,일반의원,0
7,자양골목전통시장(자양골목시장),28.662391,육류판매,22.791671,육류판매,27.067482,육류판매,24.721570,육류판매,1
8,"중곡제일시장(중곡제일골목시장, 광성시장)",23.967897,육류판매,18.605682,육류판매,19.906220,육류판매,18.972135,육류판매,1
9,화양제일골목시장,48.851831,반찬가게,50.179365,반찬가게,51.006105,반찬가게,47.223780,반찬가게,1


In [ ]:
def store_gender(raw, num): # 각 상권의 주 고객층 성별(분기별)
    hotplace2 = raw[raw['기준_분기_코드'] == num].copy() #해당 분기만
    gender = hotplace2[['상권_코드_명','남성_매출_금액', '여성_매출_금액']].copy()
    gender = gender.groupby('상권_코드_명').agg({'남성_매출_금액':'sum', '여성_매출_금액':'sum'}).reset_index(drop=False) # 각 상권별 성별의 매출액 합계
    gender['전체 매출'] = gender.sum(axis=1, numeric_only=True)
    gender['남성 매출 비율'] = gender['남성_매출_금액'] / gender['전체 매출'] * 100
    gender['여성 매출 비율'] = gender['여성_매출_금액'] / gender['전체 매출'] * 100

    gender2 = gender.drop(['남성_매출_금액','여성_매출_금액', '전체 매출'], axis=1)
    gender2['주 고객 매출 비율'] = gender2.max(axis=1, numeric_only=True)
    gender2['주 고객 성별'] = np.nan

    for i,j in enumerate(gender2['주 고객 매출 비율']):
        if gender2.loc[i, '남성 매출 비율'] == j:
            gender2.loc[i, '주 고객 성별'] = '남성'
        elif gender2.loc[i, '여성 매출 비율'] == j:
            gender2.loc[i, '주 고객 성별'] = '여성'

    gender2.drop(['남성 매출 비율', '여성 매출 비율'], axis=1, inplace=True)
    gender2.columns = ['상권_코드_명', str(num)+"분기", str(num)+"분기 주 고객"]
    return gender2

In [ ]:
df4 = concat_quart(hotplace, store_gender)
df4 = df4.merge(hotplace_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df4, "./clustering/발달상권 주 고객 성별.png", max_cols=-1, max_rows=-1)
df4

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 고객,2분기,2분기 주 고객,3분기,3분기 주 고객,4분기,4분기 주 고객,label
0,구삼강변맛길,50.423037,남성,51.185539,여성,51.471902,여성,51.710050,여성,0
1,구의사거리,55.659270,남성,53.735063,남성,57.295509,남성,51.355563,남성,0
2,구의역,58.663593,남성,57.483637,남성,56.206889,남성,59.080689,남성,0
3,아차산역,55.191935,여성,53.138195,여성,54.093615,여성,53.056186,여성,0
4,어린이대공원역,55.999401,남성,55.919614,남성,53.701996,남성,55.761594,남성,0
5,올림픽대교북단,55.378963,여성,55.370055,여성,55.542740,여성,55.799757,여성,0
6,강변역(테크노마트),63.663884,남성,58.491909,남성,61.223580,남성,55.699477,남성,1
7,건대입구역(건대),55.209930,여성,53.935153,여성,54.446074,여성,53.957267,여성,1
8,군자역,50.121924,남성,50.814191,남성,51.151227,남성,53.346379,남성,1


In [ ]:
df5 = concat_quart(nearby, store_gender)
df5 = df5.merge(nearby_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df5, "./clustering/골목상권 주 고객 성별.png", max_cols=-1, max_rows=-1)
df5

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 고객,2분기,2분기 주 고객,3분기,3분기 주 고객,4분기,4분기 주 고객,label
0,화양사거리,60.165578,남성,63.743073,남성,62.103152,남성,60.115517,남성,0
1,용마초등학교,53.074470,남성,51.809474,남성,54.559932,남성,56.515813,남성,0
2,자양4동주민센터,56.963049,여성,52.655477,남성,52.749194,남성,53.604098,여성,0
3,자양한강도서관,61.151789,남성,55.835609,남성,57.340716,남성,58.459474,남성,0
4,능동우편취급국,63.664859,남성,67.777314,남성,62.676694,남성,64.166112,남성,0
5,구의역 4번,50.013621,여성,53.799209,여성,53.624888,남성,52.900793,여성,0
6,구의1동주민센터,59.762291,남성,59.836695,남성,59.000982,남성,59.321895,남성,0
7,중곡3동주민센터,56.644333,남성,50.797994,남성,50.169674,남성,53.748747,남성,0
8,아차산역 1번,56.854874,남성,59.949117,남성,60.157702,남성,57.045521,남성,0
9,광진정보도서관,59.687079,남성,58.407000,남성,56.821832,남성,58.219025,남성,0


In [ ]:
df6 = concat_quart(market, store_gender)
df6 = df6.merge(market_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df6, "./clustering/전통시장 주 고객 성별.png", max_cols=-1, max_rows=-1)
df6

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 고객,2분기,2분기 주 고객,3분기,3분기 주 고객,4분기,4분기 주 고객,label
0,노룬산시장(노룬산골목시장),56.370702,여성,57.634032,여성,55.420857,여성,57.060837,여성,0
1,능동로골목시장,55.601541,남성,53.127556,남성,53.286809,남성,51.695571,남성,0
2,면곡골목시장,53.401124,여성,57.751899,남성,52.916329,여성,55.661753,여성,0
3,신성시장(신성골목시장),54.463209,여성,54.953843,여성,56.143915,여성,55.380599,여성,0
4,영동교골목시장,51.403291,여성,57.086052,남성,52.076742,남성,53.998508,여성,0
5,조양시장,71.561219,여성,79.715760,여성,69.565112,여성,54.853049,여성,0
6,한아름시장,55.410003,남성,56.016662,남성,53.137588,남성,51.883017,남성,0
7,자양골목전통시장(자양골목시장),55.572849,여성,55.178339,여성,55.793390,여성,58.789852,여성,1
8,"중곡제일시장(중곡제일골목시장, 광성시장)",56.875208,여성,56.242368,여성,57.024224,여성,56.231542,여성,1
9,화양제일골목시장,50.702298,여성,53.269241,남성,55.186137,남성,53.396564,여성,1


In [ ]:
def store_age(raw, num): # 각 상권의 주 고객층 연령대(분기별)
    hotplace2 = raw[raw['기준_분기_코드'] == num].copy() #해당 분기만
    age = hotplace2[['상권_코드_명','연령대_10_매출_금액', '연령대_20_매출_금액', '연령대_30_매출_금액', '연령대_40_매출_금액', '연령대_50_매출_금액', '연령대_60_이상_매출_금액']].copy()
    age = age.groupby('상권_코드_명').agg({'연령대_10_매출_금액':'sum', '연령대_20_매출_금액':'sum', '연령대_30_매출_금액':'sum', '연령대_40_매출_금액':'sum', '연령대_50_매출_금액':'sum', '연령대_60_이상_매출_금액':'sum'}).reset_index(drop=False) # 각 상권별 성별의 매출액 합계
    age['전체 매출'] = age.sum(axis=1, numeric_only=True)
    age.columns = ['상권_코드_명', '10대', '20대', '30대', '40대', '50대', '60대 이상', '전체 매출']

    col = list(age.columns)
    for c in col: # ratio 열 생성
        if (c != "전체 매출") and (c != "상권_코드_명"):
            age[f'ratio_{c}'] = age[c] / age['전체 매출'] * 100


    age = age.drop(['10대', '20대', '30대', '40대', '50대', '60대 이상', '전체 매출'], axis=1)
    age['주 고객 매출 비율'] = age.max(axis=1, numeric_only=True)
    age['주 고객 연령대'] = np.nan


    col = list(age.columns)
    for i in range(0,len(age)):
        for c in col:
            if ("주 고객" not in c) and (c != "상권_코드_명"):
                if age.loc[i, c] == age.loc[i, '주 고객 매출 비율']: #매출 최대 비율에 해당한다면
                    age.loc[i, '주 고객 연령대'] = c #매출 최대 업종에 컬럼명 삽입

    new_df = age[['상권_코드_명', '주 고객 매출 비율', '주 고객 연령대']]
    for i,j in enumerate(new_df['주 고객 연령대']): #연령대만 남기기
        new_df.loc[i, '주 고객 연령대'] = j.split("_")[1]
    new_df.columns = ['상권_코드_명', str(num)+"분기", str(num)+"분기 주 고객 연령대"]
    return new_df

In [ ]:
df7 = concat_quart(hotplace, store_age)
df7 = df7.merge(hotplace_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df7, "./clustering/발달상권 주 고객 연령대.png", max_cols=-1, max_rows=-1)
df7

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 고객 연령대,2분기,2분기 주 고객 연령대,3분기,3분기 주 고객 연령대,4분기,4분기 주 고객 연령대,label
0,구삼강변맛길,25.870133,50대,27.273789,40대,27.760486,40대,27.199022,40대,0
1,구의사거리,31.277002,60대 이상,30.886958,60대 이상,31.268187,50대,29.227480,60대 이상,0
2,구의역,23.832437,50대,22.519774,30대,22.235798,30대,21.872785,50대,0
3,아차산역,25.113494,60대 이상,25.623875,60대 이상,24.295713,50대,23.425841,50대,0
4,어린이대공원역,33.084450,20대,35.593213,20대,32.333306,20대,34.639419,20대,0
5,올림픽대교북단,41.737239,40대,41.328745,40대,42.856281,40대,43.632385,40대,0
6,강변역(테크노마트),25.959925,50대,26.663049,40대,27.281991,40대,25.709191,40대,1
7,건대입구역(건대),39.451404,20대,41.380367,20대,41.132434,20대,40.130753,20대,1
8,군자역,23.530308,30대,22.377967,30대,22.753940,30대,23.518389,30대,1


In [ ]:
df8 = concat_quart(nearby, store_age)
df8 = df8.merge(nearby_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df8, "./clustering/골목상권 주 고객 연령대.png", max_cols=-1, max_rows=-1)
df8

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 고객 연령대,2분기,2분기 주 고객 연령대,3분기,3분기 주 고객 연령대,4분기,4분기 주 고객 연령대,label
0,화양사거리,39.142498,20대,39.125382,20대,38.842330,20대,39.349337,20대,0
1,용마초등학교,26.077632,50대,26.502593,50대,25.663121,50대,27.623805,50대,0
2,자양4동주민센터,32.341412,50대,34.156370,50대,36.835416,50대,34.452663,50대,0
3,자양한강도서관,24.467017,40대,25.154543,50대,25.988815,50대,25.935883,50대,0
4,능동우편취급국,27.506474,30대,32.770330,40대,26.527541,30대,25.702098,30대,0
5,구의역 4번,29.119992,60대 이상,28.157769,60대 이상,27.529551,40대,32.391742,60대 이상,0
6,구의1동주민센터,30.547169,40대,26.502695,30대,30.843721,40대,28.519180,40대,0
7,중곡3동주민센터,29.189128,50대,24.263816,40대,23.687344,50대,25.226112,50대,0
8,아차산역 1번,29.573931,50대,31.857896,50대,31.534023,50대,26.678645,60대 이상,0
9,광진정보도서관,27.639374,50대,31.212575,50대,29.883267,40대,30.851101,40대,0


In [ ]:
df9 = concat_quart(market, store_age)
df9 = df9.merge(market_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df9, "./clustering/전통시장 주 고객 연령대.png", max_cols=-1, max_rows=-1)
df9

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 고객 연령대,2분기,2분기 주 고객 연령대,3분기,3분기 주 고객 연령대,4분기,4분기 주 고객 연령대,label
0,노룬산시장(노룬산골목시장),35.631607,60대 이상,35.484842,50대,37.058679,50대,39.606395,60대 이상,0
1,능동로골목시장,29.289767,50대,28.961459,50대,33.062588,50대,30.917554,50대,0
2,면곡골목시장,33.211961,50대,34.298969,60대 이상,32.277258,50대,31.386956,50대,0
3,신성시장(신성골목시장),33.878660,60대 이상,33.909489,60대 이상,34.669054,60대 이상,38.049761,60대 이상,0
4,영동교골목시장,31.974304,50대,36.134347,60대 이상,36.058520,50대,46.884146,50대,0
5,조양시장,33.771544,20대,34.511894,50대,35.194692,20대,36.999617,20대,0
6,한아름시장,40.000690,20대,35.500893,20대,41.914888,20대,37.254888,20대,0
7,자양골목전통시장(자양골목시장),32.629663,60대 이상,31.272737,60대 이상,33.006321,60대 이상,35.230332,60대 이상,1
8,"중곡제일시장(중곡제일골목시장, 광성시장)",29.584249,50대,30.458617,50대,30.286000,60대 이상,30.999220,50대,1
9,화양제일골목시장,24.051307,50대,24.608588,20대,26.099403,30대,26.124362,50대,1


In [ ]:
def store_day(raw, num): # 각 상권의 주 매출 요일 분석
    hotplace2 = raw[raw['기준_분기_코드'] == num].copy() #해당 분기만
    day = hotplace2[['상권_코드_명','월요일_매출_금액', '화요일_매출_금액', '수요일_매출_금액', '목요일_매출_금액', '금요일_매출_금액', '토요일_매출_금액', '일요일_매출_금액']].copy()
    day = day.groupby('상권_코드_명').sum().reset_index(drop=False) # 각 상권별 요일의 매출액 합계

    day['전체 매출'] = day.sum(axis=1, numeric_only=True)
    day.columns = ['상권_코드_명', '월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일', '전체 매출']

    col = list(day.columns)
    for c in col: # ratio 열 생성
        if (c != "전체 매출") and (c != "상권_코드_명"):
            day[f'ratio_{c}'] = day[c] / day['전체 매출'] * 100

    day = day.drop(['월요일', '화요일', '수요일', '목요일', '금요일', '토요일', '일요일', '전체 매출'], axis=1)
    day['주 매출 요일 비율'] = day.max(axis=1, numeric_only=True)
    day['주 매출 요일'] = np.nan

    col = list(day.columns)
    for i in range(0,len(day)):
        for c in col:
            if ("ratio" in c):
                if day.loc[i, c] == day.loc[i, '주 매출 요일 비율']: #매출 최대 비율에 해당한다면
                    day.loc[i, '주 매출 요일'] = c #주 매출 요일에 컬럼명 삽입

    new_df = day[['상권_코드_명', '주 매출 요일 비율', '주 매출 요일']]
    for i,j in enumerate(new_df['주 매출 요일']): #연령대만 남기기
        new_df.loc[i, '주 매출 요일'] = j.split("_")[1]
    new_df.columns = ['상권_코드_명', str(num)+"분기", str(num)+"분기 주 매출 요일"]
    return new_df

In [ ]:
df10 = concat_quart(hotplace, store_day)
df10= df10.merge(hotplace_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df10, "./clustering/발달상권 주 매출 요일.png", max_cols=-1, max_rows=-1)
df10

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 매출 요일,2분기,2분기 주 매출 요일,3분기,3분기 주 매출 요일,4분기,4분기 주 매출 요일,label
0,구삼강변맛길,16.017380,목요일,16.055522,토요일,17.319998,금요일,16.782617,토요일,0
1,구의사거리,18.688050,월요일,18.838098,화요일,19.745514,수요일,19.427189,화요일,0
2,구의역,16.412813,금요일,16.811795,금요일,17.271718,금요일,16.712402,토요일,0
3,아차산역,16.751775,월요일,17.860491,금요일,17.466085,금요일,18.026467,토요일,0
4,어린이대공원역,16.425632,목요일,15.734277,금요일,16.659452,화요일,16.365421,토요일,0
5,올림픽대교북단,16.464965,토요일,17.212728,금요일,17.923199,금요일,16.996450,토요일,0
6,강변역(테크노마트),18.937890,토요일,16.272292,토요일,17.604967,수요일,16.976644,목요일,1
7,건대입구역(건대),17.294122,토요일,18.420389,토요일,16.922716,금요일,18.865946,토요일,1
8,군자역,16.579664,금요일,16.626169,금요일,17.051852,금요일,18.297227,토요일,1


In [ ]:
df11 = concat_quart(nearby, store_day)
df11 = df11.merge(nearby_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df11, "./clustering/골목상권 주 매출 요일.png", max_cols=-1, max_rows=-1)
df11

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 매출 요일,2분기,2분기 주 매출 요일,3분기,3분기 주 매출 요일,4분기,4분기 주 매출 요일,label
0,화양사거리,17.579787,토요일,18.193152,토요일,16.642401,금요일,17.002342,토요일,0
1,용마초등학교,18.110071,토요일,17.184456,월요일,16.873501,금요일,18.652058,토요일,0
2,자양4동주민센터,20.946242,목요일,21.742071,화요일,21.085110,목요일,20.552366,목요일,0
3,자양한강도서관,16.969757,토요일,18.032020,토요일,16.842433,토요일,17.972806,토요일,0
4,능동우편취급국,16.335737,금요일,28.096007,금요일,15.902786,목요일,16.163673,화요일,0
5,구의역 4번,17.313306,목요일,16.202742,금요일,21.447408,월요일,17.072491,수요일,0
6,구의1동주민센터,17.529112,토요일,17.116053,토요일,17.375518,금요일,17.023171,토요일,0
7,중곡3동주민센터,19.706276,화요일,18.250467,토요일,18.190525,금요일,21.017386,토요일,0
8,아차산역 1번,17.182204,월요일,16.610006,토요일,16.507099,금요일,19.498755,토요일,0
9,광진정보도서관,20.678599,토요일,16.237446,토요일,17.187550,금요일,17.102032,토요일,0


In [ ]:
df12 = concat_quart(market, store_day)
df12 = df12.merge(market_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df12, "./clustering/전통시장 주 매출 요일.png", max_cols=-1, max_rows=-1)
df12

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 매출 요일,2분기,2분기 주 매출 요일,3분기,3분기 주 매출 요일,4분기,4분기 주 매출 요일,label
0,노룬산시장(노룬산골목시장),17.845990,토요일,22.789302,토요일,19.506512,토요일,21.493045,토요일,0
1,능동로골목시장,17.574524,토요일,20.486322,토요일,19.064612,토요일,19.020828,토요일,0
2,면곡골목시장,17.466130,월요일,21.257668,월요일,16.357478,목요일,16.822376,수요일,0
3,신성시장(신성골목시장),15.412306,월요일,16.029897,월요일,15.707685,금요일,16.937742,금요일,0
4,영동교골목시장,23.259499,토요일,22.674487,금요일,20.511636,수요일,17.732558,수요일,0
5,조양시장,22.683323,월요일,21.820917,월요일,20.018980,토요일,21.829795,금요일,0
6,한아름시장,18.935343,월요일,18.120107,토요일,19.229136,월요일,18.753192,금요일,0
7,자양골목전통시장(자양골목시장),20.482394,토요일,19.613265,토요일,18.704517,금요일,19.725530,토요일,1
8,"중곡제일시장(중곡제일골목시장, 광성시장)",19.630720,토요일,17.259262,토요일,19.432552,금요일,19.635312,토요일,1
9,화양제일골목시장,17.484254,월요일,18.470460,토요일,18.631188,금요일,17.892280,수요일,1


In [ ]:
def store_time(raw, num): # 각 상권의 주 매출 시간대 분석
    hotplace2 = raw[raw['기준_분기_코드'] == num].copy() #해당 분기만
    time = hotplace2[['상권_코드_명','시간대_00~06_매출_금액', '시간대_06~11_매출_금액', '시간대_11~14_매출_금액', '시간대_14~17_매출_금액', '시간대_17~21_매출_금액', '시간대_21~24_매출_금액']].copy()
    time = time.groupby('상권_코드_명').sum().reset_index(drop=False) # 각 상권별 시간대의 매출액 합계

    time['전체 매출'] = time.sum(axis=1, numeric_only=True)
    time.columns = ['상권_코드_명', '새벽', '오전', '점심', '오후', '저녁', '밤', '전체 매출']

    col = list(time.columns)
    for c in col: # ratio 열 생성
        if (c != "전체 매출") and (c != "상권_코드_명"):
            time[f'ratio_{c}'] = time[c] / time['전체 매출'] * 100

    time = time.drop(['새벽', '오전', '점심', '오후', '저녁', '밤', '전체 매출'], axis=1)
    time['주 매출 시간대 비율'] = time.max(axis=1, numeric_only=True)
    time['주 매출 시간대'] = np.nan

    col = list(time.columns)
    for i in range(0,len(time)):
        for c in col:
            if ("ratio" in c):
                if time.loc[i, c] == time.loc[i, '주 매출 시간대 비율']: #매출 최대 비율에 해당한다면
                    time.loc[i, '주 매출 시간대'] = c #주 매출 요일에 컬럼명 삽입

    new_df = time[['상권_코드_명', '주 매출 시간대 비율', '주 매출 시간대']]
    for i,j in enumerate(new_df['주 매출 시간대']): #시간대만 남기기
        new_df.loc[i, '주 매출 시간대'] = j.split("_")[1]
    new_df.columns = ['상권_코드_명', str(num)+"분기", str(num)+"분기 주 매출 시간대"]
    return new_df

In [ ]:
df13 = concat_quart(hotplace, store_time)
df13 = df13.merge(hotplace_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df13, "./clustering/발달상권 주 매출 시간대.png", max_cols=-1, max_rows=-1)
df13

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 매출 시간대,2분기,2분기 주 매출 시간대,3분기,3분기 주 매출 시간대,4분기,4분기 주 매출 시간대,label
0,구삼강변맛길,33.569439,저녁,31.699841,저녁,31.311035,저녁,31.748636,저녁,0
1,구의사거리,28.994541,점심,26.063286,저녁,26.208603,점심,28.090867,오후,0
2,구의역,35.165612,저녁,31.165215,저녁,31.313746,저녁,31.889321,저녁,0
3,아차산역,29.892708,저녁,35.940462,저녁,29.741119,저녁,29.348345,저녁,0
4,어린이대공원역,40.345794,저녁,34.583451,저녁,33.407937,저녁,34.485180,저녁,0
5,올림픽대교북단,31.380101,저녁,32.317723,저녁,33.114867,저녁,32.250554,저녁,0
6,강변역(테크노마트),38.701967,오후,37.096348,오후,38.504650,오후,40.339616,오후,1
7,건대입구역(건대),40.625491,저녁,34.263726,저녁,34.019837,저녁,33.929604,저녁,1
8,군자역,33.954299,저녁,32.834805,저녁,31.887337,저녁,31.490215,저녁,1


In [ ]:
df14 = concat_quart(nearby, store_time)
df14 = df14.merge(nearby_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df14, "./clustering/골목상권 주 매출 시간대.png", max_cols=-1, max_rows=-1)
df14

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 매출 시간대,2분기,2분기 주 매출 시간대,3분기,3분기 주 매출 시간대,4분기,4분기 주 매출 시간대,label
0,화양사거리,37.160058,저녁,31.705699,저녁,31.622951,저녁,34.218492,저녁,0
1,용마초등학교,31.267379,저녁,30.566025,저녁,30.613518,저녁,29.439224,저녁,0
2,자양4동주민센터,39.954389,오후,29.141637,오후,25.093880,오후,29.984989,오후,0
3,자양한강도서관,36.442043,저녁,35.390123,저녁,38.048755,저녁,37.278914,저녁,0
4,능동우편취급국,35.813243,저녁,32.033015,저녁,35.213114,저녁,33.889021,저녁,0
5,구의역 4번,29.370559,오후,30.515392,저녁,27.423963,점심,28.323275,오후,0
6,구의1동주민센터,40.671210,저녁,34.530905,저녁,32.985638,저녁,33.953233,저녁,0
7,중곡3동주민센터,28.973027,오후,27.821285,저녁,30.252147,저녁,24.308892,저녁,0
8,아차산역 1번,40.696285,저녁,36.074403,저녁,32.927098,저녁,37.046065,저녁,0
9,광진정보도서관,32.168005,점심,29.660550,점심,32.959620,저녁,31.908861,저녁,0


In [ ]:
df15 = concat_quart(market, store_time)
df15 = df15.merge(market_standard, how='inner', on='상권_코드_명').sort_values(by='label').reset_index(drop=True)
dfi.export(df15, "./clustering/전통시장 주 매출 시간대.png", max_cols=-1, max_rows=-1)
df15

C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe
C:\Program Files (x86)\Google\Chrome\Application\chrome.exe


,상권_코드_명,1분기,1분기 주 매출 시간대,2분기,2분기 주 매출 시간대,3분기,3분기 주 매출 시간대,4분기,4분기 주 매출 시간대,label
0,노룬산시장(노룬산골목시장),32.921013,오후,34.534821,저녁,32.823861,저녁,34.788430,오후,0
1,능동로골목시장,43.089343,저녁,43.444492,저녁,39.697537,저녁,43.277546,저녁,0
2,면곡골목시장,41.043528,저녁,38.105952,저녁,38.695748,저녁,36.245195,저녁,0
3,신성시장(신성골목시장),32.066630,저녁,33.688740,저녁,31.267190,저녁,31.289974,오후,0
4,영동교골목시장,35.617912,저녁,34.936698,저녁,35.312938,저녁,33.067401,오후,0
5,조양시장,33.753648,저녁,28.454845,오전,33.028395,저녁,43.098769,오후,0
6,한아름시장,33.087978,저녁,30.898188,저녁,33.222401,저녁,36.171253,저녁,0
7,자양골목전통시장(자양골목시장),34.429630,저녁,35.724003,저녁,34.268970,저녁,34.654706,오후,1
8,"중곡제일시장(중곡제일골목시장, 광성시장)",31.413345,오후,30.258686,저녁,29.723500,저녁,28.409384,저녁,1
9,화양제일골목시장,44.441025,저녁,41.285226,저녁,37.160129,저녁,32.254219,저녁,1
